In [6]:
# Global config and token
import requests
import mysql.connector
from mysql.connector import Error

# MySQL configuration
DB_CONFIG = {
    'host': '103.76.231.90',
    'user': 'cqpplzym_home',
    'password': 'home@1234567890',
    'database': 'cqpplzym_xyz'
}


In [95]:
BEARER_TOKEN = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpbmRpdmlkdWFsVHlwZSI6Ik1FTUJFUiIsInVzZXJfbmFtZSI6ImFyanVuQG1pZ29vLmluIiwiY29uY2VwdCI6IkNPTk5FQ1QiLCJpbmRpdmlkdWFsSWQiOjk3Mzg4MjAsImF1dGhvcml0aWVzIjpbIjY4NGQ3N2ZlYjBlOWRlNGY1ZWMwN2JkZCJdLCJjbGllbnRfaWQiOiJJREVOVElUWV9QT1JUQUwiLCJsb2NhbGVDb2RlIjoiZW5fSU4iLCJzY29wZSI6WyJjb3JlIiwicHVibGljX2NoYXB0ZXJfZGV0YWlscyIsImdyb3VwcyIsInNlYXJjaCIsInRpcHMiLCJwdWJsaWNfdHJhbnNsYXRpb25zIiwibWVtYmVyIiwib25saW5lX2FwcGxpY2F0aW9ucyIsInNvY2lhbCIsInB1YmxpY19wb3J0YWwiXSwiYWNjZXB0ZWRUb1MiOls4XSwiaWQiOiIyMDU3MjIwIiwiZXhwIjoxNzQ5OTE0OTYzLCJyb2xlR3JvdXBzIjpbIk1FTUJFUiJdLCJqdGkiOiI1M2VmYjY3Zi03MmY1LTQwYjAtOThmZC00YzA0OTE4MmJmMWIiLCJrZXkiOiI2ODRkNzdmZWIwZTlkZTRmNWVjMDdiZGQifQ.--cpWX_uftkwgjg4PAtA4k-f6bA0A4jeayt3ASM9LzQ"
API_URL = "https://api.bniconnectglobal.com/connect-search-api/search/member/advanced"
primary_id = 79
speciality_ids = [791120, 792150, 792160, 799000]
headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}",
    "Content-Type": "application/json"
}

# === PAYLOAD TEMPLATE ===
base_payload = {
    "search_tags": "",
    "country": "India",
    "first_name": None,
    "city": "",
    "last_name": None,
    "state": None,
    "company_name": None,
    "category_id": primary_id,
    "speciality_id": "",
    "locale_code": "en_IN",
    "concept_id": 1,
    "page_no": 1,
    "per_page": 100
}


In [47]:
def fetch_all_profiles(speciality_id):
    all_profiles = []
    page = 1
    total_pages = 1

    while page <= total_pages:
        payload = base_payload.copy()
        payload["page_no"] = page
        payload["speciality_id"] = speciality_id
        payload["category_id"] = primary_id

        try:
            response = requests.post(API_URL, headers=headers, json=payload)
            response.raise_for_status()
            json_data = response.json()

            content = json_data.get("content", {})
            profiles = content.get("search_results") or []

            all_profiles.extend(profiles)
            total_pages = content.get("total_pages", 1)

            print(f"Fetched page {page}/{total_pages} for speciality_id {speciality_id} with {len(profiles)} records.")
            page += 1

        except Exception as e:
            print(f"Error fetching data for speciality_id {speciality_id}, page {page}: {e}")
            break

    return all_profiles


In [41]:
def insert_profile(profile_data, category_id, speciality_id):
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        cursor = connection.cursor()

        insert_query = """
            INSERT INTO profile (
                profileid, coordinates, country, city, state, category, speciality,
                score, rank, connected, user_id, first_name, last_name,
                chapter_name, company_name, profile_image_id, profile_url,
                chapter_id, membership_status_id, membership_status,
                category_id, speciality_id
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                coordinates=VALUES(coordinates),
                country=VALUES(country),
                city=VALUES(city),
                state=VALUES(state),
                category=VALUES(category),
                speciality=VALUES(speciality),
                score=VALUES(score),
                rank=VALUES(rank),
                connected=VALUES(connected),
                first_name=VALUES(first_name),
                last_name=VALUES(last_name),
                chapter_name=VALUES(chapter_name),
                company_name=VALUES(company_name),
                profile_image_id=VALUES(profile_image_id),
                profile_url=VALUES(profile_url),
                chapter_id=VALUES(chapter_id),
                membership_status_id=VALUES(membership_status_id),
                membership_status=VALUES(membership_status),
                category_id=VALUES(category_id),
                speciality_id=VALUES(speciality_id)
        """

        data = (
            profile_data["user_id"],
            profile_data.get("coordinates"),
            profile_data.get("country"),
            profile_data.get("city"),
            profile_data.get("state"),
            profile_data.get("category"),
            profile_data.get("speciality"),
            profile_data.get("score"),
            profile_data.get("rank"),
            profile_data.get("connected"),
            profile_data["user_id"],
            profile_data.get("first_name"),
            profile_data.get("last_name"),
            profile_data.get("chapter_name"),
            profile_data.get("company_name"),
            profile_data.get("profile_image_id"),
            profile_data.get("profile_url"),
            profile_data.get("chapter_id"),
            profile_data.get("membership_status_id"),
            profile_data.get("membership_status"),
            category_id,
            speciality_id
        )

        cursor.execute(insert_query, data)
        connection.commit()
    except mysql.connector.Error as err:
        print("Insert error:", err)
    finally:
        cursor.close()
        connection.close()


In [3]:
def get_total_rows():
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        cursor = connection.cursor()
        cursor.execute("SELECT COUNT(*) FROM profile")
        total_rows = cursor.fetchone()[0]
        print(f"Total rows in profile table: {total_rows}")
        cursor.close()
        connection.close()
    except Error as e:
        print("Error while counting rows:", e)


In [96]:
total_inserted = 0

for speciality_id in speciality_ids:
    profiles = fetch_all_profiles(speciality_id)
    for profile in profiles:
       insert_profile(profile, category_id=primary_id, speciality_id=speciality_id)
    total_inserted += len(profiles)

print(f"Total profiles inserted/updated across all specialities: {total_inserted}")
get_total_rows()


Fetched page 1/2 for speciality_id 791120 with 100 records.
Fetched page 2/2 for speciality_id 791120 with 2 records.
Fetched page 1/3 for speciality_id 792150 with 100 records.
Fetched page 2/3 for speciality_id 792150 with 100 records.
Fetched page 3/3 for speciality_id 792150 with 59 records.
Fetched page 1/9 for speciality_id 792160 with 100 records.
Fetched page 2/9 for speciality_id 792160 with 100 records.
Fetched page 3/9 for speciality_id 792160 with 100 records.
Fetched page 4/9 for speciality_id 792160 with 100 records.
Fetched page 5/9 for speciality_id 792160 with 100 records.
Fetched page 6/9 for speciality_id 792160 with 100 records.
Fetched page 7/9 for speciality_id 792160 with 100 records.
Fetched page 8/9 for speciality_id 792160 with 100 records.
Fetched page 9/9 for speciality_id 792160 with 6 records.
Fetched page 1/5 for speciality_id 799000 with 100 records.
Fetched page 2/5 for speciality_id 799000 with 100 records.
Fetched page 3/5 for speciality_id 799000 wit

In [7]:
get_total_rows()

Total rows in profile table: 67571


In [ ]:
import re
import time
import datetime
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc
from selenium.webdriver.support import expected_conditions as EC
# 🛠️ MySQL configuration
DB_CONFIG = {
    'host': '103.76.231.90',
    'user': 'cqpplzym_home',
    'password': 'home@1234567890',
    'database': 'cqpplzym_xyz'
}
LOG_FILE = "bni_uuid_log.txt"
def log(message):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{timestamp}] {message}\n")
    print(message)
chrome_path = r"C:\Users\BAPS\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# 🔐 Credentials
USERNAME = "arjun@migoo.in"
PASSWORD = "Arjun@108"

# 🌐 URLs
LOGIN_URL = "https://www.bniconnectglobal.com/login/"
PROFILE_URL_TEMPLATE = "https://www.bniconnectglobal.com/web/secure/networkHome?userId={user_id}"

# ✅ Login with Selenium and return browser session
def login_and_get_driver():
    chrome_options = Options()
    #chrome_options.add_argument("--headless")  # Optional: remove if you want to see UI
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

    service = Service(executable_path=chrome_path)
    driver = uc.Chrome(executable_path=chrome_path, options=chrome_options, headless=False)

    log("🌐 Opening login page...")
    driver.get(LOGIN_URL)

    wait = WebDriverWait(driver, 15)

    try:
        log("🔐 Waiting for login form...")
        log("📄📄 Final page snapshot:\n"+ driver.page_source[:1500])

        wait.until(EC.presence_of_element_located((By.NAME, "username")))

        driver.find_element(By.NAME, "username").send_keys(USERNAME)
        driver.find_element(By.NAME, "password").send_keys(PASSWORD)
        driver.find_element(By.TAG_NAME, "form").submit()

        log("⏳ Waiting for dashboard to load fully...")
        # Wait for an element that's only visible after full login (like dashboard widget or profile button)
        #wait.until(EC.presence_of_element_located((By.ID, "some-dashboard-element-id-or-class")))  # 🔁 Replace with actual

        log("✅ Selenium login successful.")
        return driver

    except Exception as e:
        log("🚫 Login error:", e)
        log("📄 Final page snapshot:\n"+ driver.page_source[:1500])
        driver.quit()
        raise Exception("❌ Selenium login failed.")


# 🔍 Fetch UUID from redirected URL
def extract_uuid_with_selenium(driver, user_id):
    try:
        profile_url = PROFILE_URL_TEMPLATE.format(user_id=user_id)
        log(f"🌐 Navigating to profile URL: {profile_url}")

        driver.get(profile_url)
        time.sleep(5)  # Wait for redirect or JS rendering

        final_url = driver.current_url
        log(f"📍 Final URL after redirect: {final_url}")

        match = re.search(r"uuId=([a-f0-9\-]+)", final_url)

        if match:
            uuid = match.group(1)
            log(f"✅ UUID extracted for user_id={user_id}: {uuid}")
            return uuid
        else:
            log(f"⚠️ UUID not found in URL for user_id={user_id}")
            log(f"🔎 Current page source preview:\n{driver.page_source[:1000]}")
            return None

    except Exception as e:
        log(f"❌ Exception during UUID extraction for user_id={user_id}: {e}")
        raise


# 💾 Update profile in DB
def update_profile_uuid(user_id, uuid):
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE profile
            SET uuid = %s, isHaveUuid = TRUE
            WHERE user_id = %s
        """, (uuid, user_id))
        conn.commit()
        log(f"✅ Updated user_id={user_id} with UUID={uuid}")
    except Exception as e:
        log(f"❌ Error updating user_id={user_id}: {e}")
    finally:
        cursor.close()
        conn.close()

# 📥 Get user_ids from DB where UUID is missing
def fetch_user_ids_without_uuid():
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute("SELECT user_id FROM profile WHERE isHaveUuid = FALSE")
        return [row[0] for row in cursor.fetchall()]
    finally:
        cursor.close()
        conn.close()

# 🚀 Execute full flow
try:
    log("🚀 Starting Selenium login process...")
    driver = login_and_get_driver()
    log("✅ Driver initialized and login successful.")

    log("📥 Fetching user IDs without UUIDs...")
    user_ids = fetch_user_ids_without_uuid()
    log(f"📊 Total users fetched: {len(user_ids)}")

    for index, user_id in enumerate(user_ids, 1):
        log(f"\n🔄 Processing {index}/{len(user_ids)}: user_id = {user_id}")

        try:
            uuid = extract_uuid_with_selenium(driver, user_id)
            log(f"🔍 UUID found: {uuid}" if uuid else "⚠️ No UUID found.")

            if uuid:
                update_profile_uuid(user_id, uuid)
            else:
                log(f"⚠️ Skipping update — no UUID for user_id={user_id}")

        except Exception as ex:
            print(f"❌ Error while processing user_id={user_id}: {ex}")

    print("🧹 Quitting driver...")
    driver.quit()
    print("✅ Driver closed. Process complete.")

except Exception as login_error:
    print("❌ Exception occurred during login or setup:")
    print(login_error)



🚀 Starting Selenium login process...
🌐 Opening login page...
🔐 Waiting for login form...
📄📄 Final page snapshot:
<html lang="en"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><link rel="icon" href="data:,"><link rel="stylesheet" href="https://fonts.googleapis.com/icon?family=Material+Icons"><style type="text/css">@font-face{font-family:Notification;src:url(/login/c0d3c94cd6112550c51d7d1ed13b9da1.eot);src:url(/login/c0d3c94cd6112550c51d7d1ed13b9da1.eot?#iefixs3g3t9) format("embedded-opentype"),url(/login/651771e1df95c807c99608188d0a4287.woff) format("woff"),url(/login/0b4ac1dc75df35e169b70d7719afe4cc.ttf) format("truetype"),url(/login/5bee74caefdf9d0a834915f6c8eeb259.svg#notification) format("svg");font-weight:400;font-style:normal}.notification-container{box-sizing:border-box;position:fixed;top:0;right:0;z-index:999999;width:320px;padding:0 15px;max-height:calc(100% - 30px);overflow-x:hidden;overflow-y:auto}.notification{box-sizing:borde

KeyboardInterrupt: 

In [ ]:
pip install selenium undetected-chromedriver mysql-connector-python requests